In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import dependencis
import json
import pandas as pd
from pandas.io.json import json_normalize  
import numpy as np
import re
import time

import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import psycopg2
from config import db_password

# Dataset
Load datasets into Pandas

In [3]:
# Read files 
movies_metadata = pd.read_csv(".\Resources\movies_metadata.csv") 
len(movies_metadata)

45463

In [4]:
movies_rawdata = pd.read_csv(".\Resources\movies_rawdata.csv") 
len(movies_rawdata)

4226

In [5]:
movies_metadata.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
39305,False,NaN,0,"[{'id': 27, 'name': 'Horror'}, {'id': 10749, '...",NaN,61689,tt0080016,en,Thirst,The descendant of Elizabeth Bathory is abducte...,...,9/28/1979,0.0,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,This ancient Evil is now a modern industry.,Thirst,False,4.9,14.0
18066,False,NaN,17000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",http://www.thedoublemovie.com/,73499,tt1646980,en,The Double,The mysterious murder of a US senator bearing ...,...,10/27/2011,3696232.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Keep your enemies close,The Double,False,5.8,226.0
22500,False,"{'id': 386410, 'name': 'Dragon Ball Collection...",0,"[{'id': 28, 'name': 'Action'}, {'id': 16, 'nam...",NaN,39148,tt0142250,ja,Doragon Bōru: Saikyō e no Michi,A retelling of Dragon Ball's origin with a dif...,...,3/2/1996,0.0,74.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Dragon Ball: The Path to Power,False,7.0,35.0
18682,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,53403,tt0005810,en,A Night Out,"After a visit to a pub, Charlie and Ben cause ...",...,2/15/1915,0.0,33.0,[],Released,NaN,A Night Out,False,5.5,10.0
16129,False,NaN,15000000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",http://www.kingsspeech.com/,45269,tt1504320,en,The King's Speech,The King's Speech tells the story of the man w...,...,9/6/2010,414211549.0,118.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Find your voice.,The King's Speech,False,7.6,2817.0


In [6]:
# Drop NAs from 'Release_date' column
movies_metadata.dropna(subset=['release_date'],inplace=True)

In [7]:
# Lead file into a Dataframe
movies_metadata_df = pd.DataFrame(movies_metadata)
movies_metadata_df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,2/10/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45457,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,30840,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",...,5/13/1991,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Robin Hood,False,5.7,26.0
45459,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,11/17/2011,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45460,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,8/1/2003,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45461,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,10/21/1917,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


# Clean 'genres' column and filter word search

In [8]:
movies_metadata_df = pd.DataFrame(movies_metadata)
for index, row in movies_metadata_df.iterrows():
    #output=json.loads(row[3])
    new_genres = row[3].replace("'id'", "").replace(", 'name':", "").replace("/", "")
    remove_digits = new_genres.maketrans('', '', '0123456789')
    new_genres = new_genres.translate(remove_digits)
    new_genres = new_genres.replace("[","").replace("]","").replace("{","").replace("'","").replace(" ","").replace("}","").replace("id", "").replace(":", "")
    #st = st[st.find(',') : 9]
    print(new_genres)
    movies_metadata_df.at[index,'genres'] = new_genres
    #row.replace(st, inplace=True)
    
    #print(st.substring("[","").replace("{",""))

Animation,Comedy,Family
Adventure,Fantasy,Family
Romance,Comedy
Comedy,Drama,Romance
Comedy
Action,Crime,Drama,Thriller
Comedy,Romance
Action,Adventure,Drama,Family
Action,Adventure,Thriller
Adventure,Action,Thriller
Comedy,Drama,Romance
Comedy,Horror
Family,Animation,Adventure
History,Drama
Action,Adventure
Drama,Crime
Drama,Romance
Crime,Comedy
Crime,Comedy,Adventure
Action,Comedy,Crime
Comedy,Thriller,Crime
Drama,Thriller
Action,Adventure,Crime,Thriller
Drama,Fantasy,ScienceFiction,Thriller
Drama,Romance
Drama
Comedy,Drama,Family
Drama,Romance
Fantasy,ScienceFiction,Adventure
Drama,Crime
Drama,Crime
ScienceFiction,Thriller,Mystery
Romance,Adventure
Fantasy,Drama,Comedy,Family
History,Drama,Romance
Drama
Adventure,History,Drama,Family
Comedy,Family,Romance
Comedy,Drama,Romance
Drama
Drama,War
Action,Crime,Drama,History
Drama,Romance
Action,Fantasy
Fantasy,Drama,Comedy,Thriller
Drama,Romance
Crime,Mystery,Thriller
Adventure,Animation,Drama,Family
Drama,Romance
Drama,Crime,Thriller
Act

Adventure,Comedy
Crime,Drama
Adventure,Drama
Crime,Mystery,Thriller

Drama,Music,Romance
Crime,Thriller
Drama,War,Action
Comedy
Thriller,Mystery
Drama,Music
Romance,Crime,Drama,Thriller
Crime,Drama,Mystery
Drama,Romance
ScienceFiction,Adventure,Family,Fantasy
Comedy
Fantasy,Drama,Family
Adventure
Action,Romance,War
Action,Comedy
Drama
Drama
Horror
Documentary
Comedy
Crime,Drama
Documentary
Comedy
Comedy
Crime,Drama
Comedy,Horror
Drama
Drama
Action,Adventure,Drama,Western
Drama
Drama
Comedy
Documentary
Drama
Mystery,Comedy,Crime
Comedy,Fantasy,Family
Adventure,Action,Thriller,ScienceFiction
Horror
ScienceFiction,Action
Drama,Horror
Drama
Drama
Drama
Comedy
Adventure,Comedy,Fantasy
Romance,Drama

Drama
Drama
Comedy,Crime
Documentary
Documentary

Documentary
Animation,Comedy,Family
Documentary
Mystery,Crime,Drama,History,Romance
Animation,Drama,Fantasy
Drama,Thriller
Crime
Thriller,Crime
Drama
Drama


Comedy
Drama,History,War
Drama,Comedy
Drama,Comedy
Comedy,Drama
Drama
Comedy
Horror
Docu

Animation,Drama,Family
Adventure,Animation,Drama
Fantasy,Animation,Drama,Family
Fantasy
Drama,Family,Animation,Adventure
Adventure,Animation,Family
Adventure,Fantasy
Comedy,Romance
Comedy
Drama,Romance
Adventure,Drama
Comedy,Horror
Fantasy,Horror,Comedy
Action,Comedy
Action,Comedy
Comedy,Family
Thriller
Comedy,Drama,Romance
Comedy,Drama

Drama
Crime,Horror,Thriller
Crime,Drama,Horror,Thriller
Horror,Drama,Mystery
Drama,Family,Fantasy,Adventure
Drama,Family,Fantasy,Adventure
Comedy,Horror,ScienceFiction
Horror,ScienceFiction
Drama,Comedy
Drama,Thriller,Romance
Horror,Action
Drama,Music,Romance
Comedy
Action,Adventure,Comedy
Comedy,Drama,Romance
Comedy
Adventure,Action,Drama,Fantasy,Mystery
Fantasy,Comedy
Romance,Drama
Crime,Drama,Mystery,Thriller
Comedy,Crime,Thriller
Crime,Drama,Mystery,Thriller
Action,Drama,Mystery,Thriller
Mystery,Thriller
Crime,Drama,Mystery,Thriller
Crime,Drama,Mystery
Adventure,Crime,Drama,Mystery,Thriller
Comedy,Mystery,Thriller
Crime,Drama
Crime,Drama,Thriller
T

Action,Drama,History,War
History,Drama,Action
Western
Drama,Romance
Drama
Comedy
Comedy,Romance
Drama
Comedy,Drama,Romance
Drama
Romance,Drama,Thriller,Horror,Mystery
Mystery,Thriller
Drama,Romance
Adventure,Comedy,Crime
Drama
Drama,Romance,History
Action,Crime,Drama,Mystery,Thriller
Comedy,Drama
Drama
Drama
Drama,Family
Drama
Action,Fantasy,Horror,Mystery
Animation,Comedy,Family
Comedy,Drama
Drama
Drama,War,Romance,Western
Comedy,Drama
Horror,Thriller
Comedy
Crime,Thriller
Action,Western

Drama
Drama
Romance
Comedy,Drama
Drama
Comedy,Drama
Comedy,Drama

Drama,Comedy
Comedy,Western
Drama,History,War
Drama
Documentary
War,Action,Thriller
Drama,Romance
Action,Adventure
Drama,Comedy
Crime,Drama,Mystery
Documentary,Music
Drama,War
Comedy,Romance
Drama
Comedy
Fantasy,Drama,Crime
Drama
Comedy,Drama,Mystery
Drama
Crime,Drama,Mystery,Thriller
Drama
Action,Adventure,Family,Fantasy
Action,Drama,War
Drama,History,Romance
Comedy,ScienceFiction
Animation,Fantasy,Family,Comedy
Drama,History
Animatio

Documentary
Crime,Drama
Comedy,Family,Music,Romance
Crime,Comedy,Drama
Comedy
Horror,ScienceFiction
War
Action,Crime,Thriller
History,Comedy,Drama
Mystery,Thriller
Comedy,Drama,Romance
Crime,Comedy,Romance
Comedy,Romance

Comedy,Romance
Action,Comedy,Family,Adventure
Comedy,Romance
Drama,Thriller
Drama,Thriller
Documentary
Documentary,Foreign
Crime,Mystery,Thriller,Action
Crime,Drama
Comedy,Fantasy,ScienceFiction
Adventure,Fantasy,Animation,Action,Family
Comedy,Crime,Drama,Thriller
Crime,Drama,Mystery,Thriller
Drama,Foreign
Drama,Foreign,History
Comedy,Romance,Drama
Mystery,Adventure,Comedy,Romance,Drama
Comedy,Music
Comedy
Drama,Mystery,Romance,Thriller
Crime,Drama,Thriller
Drama,Foreign

Adventure,Comedy
Comedy
Drama,Romance
Drama,Thriller,Romance
Romance,Comedy,Drama
Romance,Drama
Drama
Drama,Fantasy,Romance
Action,Crime,Drama,Thriller
Drama,Romance
Action
Action,Adventure,Horror,Thriller
Action,Comedy,Crime
Comedy,Romance
Drama,Fantasy,Comedy,Family
Adventure,Action,Fantasy
Drama
F

...

Drama,History
Comedy
TVMovie,Horror,Comedy,Music,Thriller,ScienceFiction
Action
Drama,Mystery,Romance
Documentary
Crime,Thriller,Western,Action
Mystery,Thriller,Horror
Comedy,Romance,Drama
Drama
Horror,Action
ScienceFiction,Drama,Music
Family,Animation
Drama
Drama,Comedy
Comedy,Thriller
Drama
Comedy
Fantasy
Romance,Comedy
TVMovie,Family
TVMovie,Family
Fantasy,Drama,Comedy
Fantasy,Horror,ScienceFiction,Family
Music
Fantasy,Drama,Music,Romance,Foreign
Drama,Comedy,Romance
Comedy,Family,Fantasy
Mystery,Comedy,ScienceFiction
Drama,Crime
Drama
Animation,Drama,Western
Comedy
Drama
Thriller,Drama
Documentary
Action,Romance,Western
Action,Family,Adventure
Drama
Horror
Crime,Drama
Drama,History
Drama
Mystery
Drama,Music,Romance
TVMovie,Drama,Romance
Documentary
ScienceFiction,Thriller
Comedy
Comedy,Horror
Horror,Comedy
War,Animation,Drama
Documentary
Documentary
Crime,Comedy
Comedy,Fantasy,Family
TVMovie,Drama,Comedy,Romance,Family
Comedy,Family,Fantasy
Drama
Horror
Action,ScienceFiction



Cri

Comedy
Drama
Crime
Crime,Thriller
Drama,Horror,ScienceFiction
Animation,Fantasy,Mystery
TVMovie,Drama,Family
Comedy
Comedy
Comedy
Crime,Comedy

Documentary

Drama
Drama,History,Crime,Mystery,Thriller
Mystery,Drama,Crime
Thriller,Adventure,ScienceFiction,Mystery
Thriller,Action,Crime
Mystery,Thriller,Horror
Action,Drama,ScienceFiction
Action,Adventure,Fantasy
Comedy
Comedy,Fantasy,Horror,ScienceFiction
Crime,Drama,Thriller
Crime,Drama
Fantasy,Drama
Animation,Foreign
Drama,Documentary,Family
Adventure,Documentary,ScienceFiction
ScienceFiction
Action,ScienceFiction,Thriller
Documentary,Comedy,TVMovie
Mystery,Crime,TVMovie,Horror
Mystery,Drama,Thriller
Action,Horror,Thriller
Comedy
Romance,Drama
Animation

Drama
Romance,Comedy
Drama
Comedy

Drama,Comedy


Horror,Thriller
Drama
Horror
Drama,Horror,Romance,Foreign
Foreign,Horror,Thriller
Drama,Thriller
Comedy,Drama,Romance
Drama,Comedy
Comedy,Crime,Drama,Thriller
Documentary
Comedy,Drama
Family,Adventure,Drama
Drama


Action,Drama,Fantasy
Mu

Action,ScienceFiction
Romance,ScienceFiction
Documentary,Music
Horror,Mystery,Thriller
Drama,ScienceFiction
Drama
Thriller,Horror,Action
Documentary
Action
Action,Drama,War
Adventure,Drama,War
ScienceFiction,Animation,Action
Documentary
Drama


Comedy
Thriller
Action,ScienceFiction,Thriller
Music,Documentary
Drama,Thriller
Documentary
Action,Drama
Drama
Adventure,Comedy,Fantasy
Drama,Thriller
Comedy,ScienceFiction,Horror
Comedy,Crime,Drama
Music
Foreign,Comedy,Romance
Drama,Romance,Thriller
Documentary
Comedy,Crime
Drama
Animation,Comedy,Family
Comedy
Drama
Drama,Romance
Adventure,Animation,Comedy,Family,ScienceFiction
Drama,Romance
Comedy,Family
Drama
Adventure,Family
TVMovie,Thriller,Drama,Crime
Comedy,Adventure
Mystery,Drama,ScienceFiction
Drama,Comedy
Action,Fantasy
Adventure,Drama,History
Drama
Action,Comedy
Drama,Thriller
Action,Comedy
Romance,ScienceFiction,Thriller
Drama
Music
Animation
Thriller,Action,Adventure
Comedy,Drama,Music
Adventure
Action,Horror
Comedy,Family,Fantasy
R

Drama,Romance,Crime
Horror,Thriller
Adventure,Crime,Comedy,ScienceFiction
Romance,Action,Adventure,Crime,Drama
ScienceFiction
Drama
Horror
Action,Adventure,Romance
Drama,Horror,Adventure
Crime,Drama
Drama
Crime,Drama,Thriller
Drama
Thriller
Drama
Crime,Drama
Thriller,Action,Adventure
Drama,Romance,ScienceFiction
Action,Foreign
Horror,ScienceFiction
Drama,Horror
Horror
ScienceFiction
Romance,Fantasy,Horror
Drama,Fantasy,Horror,ScienceFiction
Family,Fantasy,Comedy,Romance
ScienceFiction,TVMovie,Action,Thriller
Action,Adventure
Action,ScienceFiction
Horror,Music

Horror,ScienceFiction
Horror
Horror
Action,ScienceFiction,Horror
Horror,Adventure,Fantasy
Action,Horror,Thriller
Adventure,Fantasy,Action
Action
Fantasy,Horror,ScienceFiction
Horror,Fantasy
Horror,ScienceFiction
Fantasy,Drama
Horror
Horror,ScienceFiction
ScienceFiction
Thriller,Romance,ScienceFiction
Drama
TVMovie,Crime,Mystery,Thriller
Drama,ScienceFiction,War
Thriller
Thriller
Horror,Thriller
Action,Foreign
Drama,Comedy
Drama
D

Comedy
Documentary,TVMovie
Comedy
Comedy
Animation
Action,Comedy,Crime
Animation,Family
Drama
Documentary

Horror,Thriller
Drama
Comedy,Drama
Drama
Comedy,Documentary
Drama
Drama,Comedy
Drama
Drama
Romance,Drama
Drama
Drama
ScienceFiction,Comedy,Fantasy
Drama,Comedy
Adventure,Family
Comedy,Drama
Crime,Comedy
Comedy,Drama
Drama
Drama
Documentary,Foreign
Drama
Comedy
Drama,Foreign
ScienceFiction,Action
Drama,Romance
Comedy,Drama
Fantasy,Family
Music,Comedy,Documentary
Drama
Comedy
Mystery
Drama,Foreign
Animation


Comedy,Animation
Drama
Comedy,Drama,Romance
Fantasy,Family
Adventure,Family,Fantasy

Drama
Music,Documentary
Romance,Drama,Fantasy
Drama
Drama
Drama,Comedy,Foreign
Documentary,Comedy
Comedy
Drama

Drama
Comedy,Drama
Documentary
Comedy,Drama
Comedy,Drama,War

Horror,Comedy,Fantasy
Crime,Drama,Thriller
Drama
Thriller,ScienceFiction
Comedy
Comedy,Fantasy,ScienceFiction
Thriller
Drama
Comedy
Fantasy,Horror,Action,Comedy,ScienceFiction,Foreign,Animation

Western,ScienceFiction
Horro

In [9]:
movies_metadata_df.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"Animation,Comedy,Family",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"Adventure,Fantasy,Family",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"Romance,Comedy",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"Comedy,Drama,Romance",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,2/10/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [10]:
# Observe the dataframe length
len(movies_metadata_df)

45376

In [11]:
# Observe the column data types
movies_metadata_df.dtypes

adult                       bool
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
id                         int64
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [12]:
# Drop NAs from 'Release_date' column
# movies_metadata.dropna(subset=['release_date'],inplace=True)

In [14]:
len(movies_metadata_df)

45376

## Prepare data for merging on 'release_year'

In [15]:
#Convert 'Release_date' from object to datetime
movies_metadata_df['release_date']=pd.to_datetime(movies_metadata_df['release_date'])

In [16]:
# Extract release year into new column
movies_metadata_df['release_year']=movies_metadata_df['release_date'].dt.year

In [17]:
# Check data types
movies_metadata_df.dtypes

adult                              bool
belongs_to_collection            object
budget                            int64
genres                           object
homepage                         object
id                                int64
imdb_id                          object
original_language                object
original_title                   object
overview                         object
popularity                      float64
poster_path                      object
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
status                           object
tagline                          object
title                            object
video                            object
vote_average                    float64
vote_count                      float64
release_year                      int64


In [18]:
movies_metadata_df['release_year'].value_counts()

2014    1974
2015    1905
2013    1889
2012    1722
2011    1667
        ... 
1883       1
2020       1
1878       1
1893       1
1874       1
Name: release_year, Length: 135, dtype: int64

In [19]:
#Convert 'Release_year' to int. This will help merging with Movies_rawdata
movies_metadata_df['release_year']=movies_metadata_df['release_year'].astype("int")

In [22]:
movies_metadata_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,release_year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"Animation,Comedy,Family",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"Adventure,Fantasy,Family",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"Romance,Comedy",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"Comedy,Drama,Romance",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


## Clean the data 
clean the data and filter unwated columns

In [23]:
# print all column names
list(movies_metadata.columns.values.tolist())

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [24]:
# Print columns
movies_metadata_df.columns.values

array(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'id', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'release_year'], dtype=object)

In [25]:
# Drop the columns that are not relevant and has more null values and 
movies_metadata_df = movies_metadata_df.drop(['belongs_to_collection'], axis=1)

In [26]:
movies_metadata_df = movies_metadata_df.drop(['homepage'], axis=1)

In [27]:
movies_metadata_df= movies_metadata_df.drop(['video'], axis=1)

In [28]:
movies_metadata_df.dtypes

adult                             bool
budget                           int64
genres                          object
id                               int64
imdb_id                         object
original_language               object
original_title                  object
overview                        object
popularity                     float64
poster_path                     object
production_companies            object
production_countries            object
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
spoken_languages                object
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
release_year                     int32
dtype: object

In [29]:
# handling missing data
print("null values: \n", end=" ")
# movies_metadata_df.isna()
movies_metadata_df.isna().sum()

null values: 
 

adult                       0
budget                      0
genres                      0
id                          0
imdb_id                    14
original_language          11
original_title              0
overview                  941
popularity                  0
poster_path               339
production_companies        0
production_countries        0
release_date                0
revenue                     0
runtime                   246
spoken_languages            0
status                     80
tagline                 24978
title                       0
vote_average                0
vote_count                  0
release_year                0
dtype: int64

In [30]:
# Print total null values for clarity
movies_metadata_df.isna().sum().sum()

26609

In [31]:
# clean_movies_metadata_df = movies_metadata_df.dropna(how='any')
# print ("new dataframe shape:", clean_movies_metadata_df.shape)


we observed that dropping nas would lead to half data reduction which can be potential for further analysis. Instead lets fill nas for only required columns


In [32]:
# fill NAs for missing values
movies_metadata_df[['imdb_id', 'original_language', 'overview', 'poster_path', 'runtime', 'status', 'tagline']] = movies_metadata_df[['imdb_id', 'original_language', 'overview', 'poster_path', 'runtime', 'status', 'tagline']].fillna(value=0)

## Duplicate Data in a Data-Frame


In [33]:
# identify unique column
movies_metadata_df.shape

(45376, 22)

In [34]:
movies_metadata_df['imdb_id'].shape

(45376,)

In [35]:
len(movies_metadata_df.imdb_id.unique())

45333

In [36]:
# Check duplicative columns
duplicated_rows_df_imdb_id= movies_metadata_df[movies_metadata_df.duplicated(['imdb_id'])]
duplicated_rows_df_imdb_id.shape

(43, 22)

In [37]:
# drop duplicates 
movies_metadata_df.drop_duplicates()
print ("shape of dataframe after dropping duplicates: ", movies_metadata_df.drop_duplicates().shape)


shape of dataframe after dropping duplicates:  (45359, 22)


## Data bucketing using popularity

divide the popularoty into given buckets

0-30 -> low </br>
 31-60 -> moderate </br>
 76-100 -> high </br>

In [38]:
op_labels = ['low', 'moderate', 'high']
category = [0.,4.,7.,10.]
movies_metadata_df['popularity_type'] = pd.cut(movies_metadata_df['popularity'], labels=op_labels, bins=category, include_lowest=False)

In [39]:
movies_metadata_df

,adult,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,popularity_type
0,False,30000000,"Animation,Comedy,Family",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,0,Toy Story,7.7,5415.0,1995,NaN
1,False,65000000,"Adventure,Fantasy,Family",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,NaN
2,False,0,"Romance,Comedy",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,NaN
3,False,16000000,"Comedy,Drama,Romance",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,low
4,False,0,Comedy,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45457,False,0,"Drama,Action,Romance",30840,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,/fQC46NglNiEMZBv5XHoyLuOWoN5.jpg,...,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,0,Robin Hood,5.7,26.0,1991,moderate
45459,False,0,Drama,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,0,Century of Birthing,9.0,3.0,2011,low
45460,False,0,"Action,Drama,Thriller",67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,3.8,6.0,2003,low
45461,False,0,,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,0.0,87.0,[],Released,0,Satan Triumphant,0.0,0.0,1917,low


In [40]:
## prepare moview raw_data

In [41]:
movies_rawdata_df = pd.DataFrame(movies_rawdata)
movies_rawdata_df

,video_id,cvt_per_day,weighted_categorical_position,weighted_horizontal_poition,import_id,release_year,genres,imdb_votes,budget,boxoffice,imdb_rating,duration_in_mins,metacritic_score,awards,mpaa,star_category
0,385504,307127.605600,1,3,lionsgate,2013,"Action,Thriller,Drama",69614,15000000,42930462,6.5,112.301017,51,other award,PG-13,1.710000
1,300175,270338.426400,1,3,lionsgate,2013,"Comedy,Crime,Thriller",46705,15000000,3301046,6.5,94.983250,41,no award,R,3.250000
2,361899,256165.867400,1,3,other,2012,"Crime,Drama",197596,26000000,37397291,7.3,115.763675,58,other award,R,2.646667
3,308314,196622.721000,3,4,lionsgate,2008,"Thriller,Drama,War,Documentary,Mystery,Action",356339,15000000,15700000,7.6,130.703583,94,Oscar,R,1.666667
4,307201,159841.652100,1,3,lionsgate,2013,"Crime,Thriller,Mystery,Documentary",46720,27220000,8551228,6.4,105.545533,37,other award,R,3.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4221,160848,9.726396,3,32,other,2010,"Comedy,Drama",14,0,0,7.8,4.311600,0,no award,NotRated,1.700000
4222,365178,7.912698,29,35,other,2013,Music,0,0,0,0.0,4.878900,0,no award,PG-13,0.000000
4223,11615,2.935691,3,27,other,2008,Comedy,505,0,0,7.1,110.350000,0,other award,NotRated,0.000000
4224,26365,2.866614,4,15,other,2003,Documentary,5,100000,0,6.0,79.633333,0,no award,NotRated,0.000000


In [42]:
movies_rawdata_df.dtypes

video_id                           int64
cvt_per_day                      float64
weighted_categorical_position      int64
weighted_horizontal_poition        int64
import_id                         object
release_year                       int64
genres                            object
imdb_votes                         int64
budget                             int64
boxoffice                          int64
imdb_rating                      float64
duration_in_mins                 float64
metacritic_score                   int64
awards                            object
mpaa                              object
star_category                    float64
dtype: object

# Merge Data